# Simple Statistics in Python

I'm going to show you how to run some simple statistics using Python.

In general, Python is very powerful for machine learning (e.g., scikit-learn, TensorFlow, etc.), while R is designed for statistics and cutting-edge statistical tools typically show up there first. That being said, all of the basic tools of a social science researcher are avaiable in Python.

In this notebook, I show you how to run some basic statistical tests and models using the [scipy stats module](https://docs.scipy.org/doc/scipy/reference/stats.html). I am assuming that you already have a working knowledge of what these statistical tests do. I am just showing you how to perform them in Python.

* Note: I personally do most of my statistical modeling in R, so I may be missing some of the tools that pure Python researchers would be aware of.



In [115]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Getting the data

I'm going to just create some random data

In [117]:
X1 = stats.norm.rvs(size = 100) # 100 random, normally distributed values
X2 = stats.norm.rvs(size = 100)
X3 = stats.norm.rvs(size = 100)
group = np.random.choice(['A','B','C'], size=100)
# Our outcome is influenced by X1, X2, and the group, plus some random noise
Y = 1.5 * X1 - 2.3 * X2 + 3 * (group == 'A') + 1.2 * (group == 'B') + stats.norm.rvs(size = 100)

# We can store these in a data frame
df = pd.DataFrame({'X1':X1,
                   'X2':X2,
                   'X3': X3,
                   'group':group,
                   'Y':Y})

## Univariate statistics

There are lots of univariate statistics we can get - mean, median, quartiles, quantiles, etc.

In [ ]:
# These all use numpy. This is the mean
np.mean(X1)

In [ ]:
# And this is how you do the same thing with data in a data frame.
# All columns are numpy arrays underneath, so this first should work for
# any of the statistics.

# Numpy way
np.mean(df.X1)

In [ ]:
# Pandas also has a number of statistics built in, which you can apply directly
df.X1.mean()

In [122]:
# For all columns
df.mean()

X1    0.049253
X2    0.131064
X3   -0.157963
Y     1.220645
dtype: float64

In [154]:
# Pandas is obviously great for doing grouping, which you often want for this
# type of statistics. "aggregate" lets you get multiple statistics

df.groupby('group').aggregate([np.mean, np.median])

X1                  X2                  X3                   Y  \
           mean    median      mean    median      mean    median      mean   
group                                                                         
A     -0.195127 -0.114133  0.155421  0.321393 -0.248183 -0.256167  2.393903   
B      0.139591  0.134866  0.073357  0.077759 -0.150291 -0.069234  1.198653   
C      0.254762  0.390598  0.156240  0.143913 -0.058018  0.007463 -0.151981   

                 
         median  
group            
A      2.470764  
B      1.540986  
C     -0.078526

In [157]:
# You can even write your own custom functions to aggregate

def mean_plus_1(array):
    array = array + 1
    return np.mean(array)

df.groupby('group').agg(mean_plus_1)

,X1,X2,X3,Y
group,,,,
A,0.804873,1.155421,0.751817,3.393903
B,1.139591,1.073357,0.849709,2.198653
C,1.254762,1.156240,0.941982,0.848019


In [61]:
# Back to just working with arrays - here's the numpy version of medians
np.median(X1)

-0.003303913596689736

In [109]:
# Quantile takes an array and a list of the quantiles you want.
# This shows the median and the quartiles
np.quantile(X1, [.25, .5, .75])

array([-1.65504907, -0.00330391,  1.6066461 ])

In [110]:
# The describe function lists a number of these
stats.describe(X1)

DescribeResult(nobs=100, minmax=(-1.8543308977911612, 2.2338785715595577), mean=-0.013540739077387403, variance=0.827555827924176, skewness=-0.017790983895410666, kurtosis=-0.559892463011074)

In [159]:
# This also works for dataframes, with a different set of stats
df.describe()

,X1,X2,X3,Y
count,100.000000,100.000000,100.000000,100.000000
mean,0.049253,0.131064,-0.157963,1.220645
std,0.933003,0.899164,0.928194,2.924822
min,-1.988523,-1.617695,-2.576751,-4.997155
25%,-0.518430,-0.507311,-0.783875,-0.730585
50%,0.147869,0.174587,-0.108387,1.286437
75%,0.653752,0.701115,0.395929,3.223490
max,2.198207,2.555347,1.889556,8.388738


## Bi-variate statistics

### Correlations
Scipy has both Pearson's coorelation and Spearman's rank correlation.

In [ ]:
# These 2 should not be correlated. 
stats.pearsonr(X1, X2)
# the first value returned is R, the second is the p-value

In [ ]:
# For pandas, you can get a correlation matrix
df.corr()

In [168]:
# Or just pass the columns you are interested in
stats.pearsonr(df.X1, df.X2)

(-0.12418847196390373, 0.2183067465174248)

In [65]:
stats.spearmanr(X1, X2)

SpearmanrResult(correlation=-0.01954995499549955, pvalue=0.8469119222123507)

In [113]:
# These should be correlated, on the other hand
stats.pearsonr(X1, Y)

(0.4680999305382044, 9.062809332329399e-07)

In [67]:
stats.spearmanr(X1, Y)

SpearmanrResult(correlation=0.453981398139814, pvalue=2.095768959903901e-06)

### T-tests

In [ ]:
# T-tests test whether 2 distributions have the same mean.

# X1-X3 all should have the same mean, but Y should differ

In [160]:
stats.ttest_ind(X1, X2)

Ttest_indResult(statistic=-0.6313759261334511, pvalue=0.5285229634446339)

In [114]:
stats.ttest_ind(X3, Y)

Ttest_indResult(statistic=-4.3256364674048084, pvalue=2.409995406541033e-05)

## Multivariate statistics

### Chi-squared test

These test whether the frequency of something occurring by group is independent. So, we'll need to change Y into something that has a frequency.

The following code will produce the 2 rows of a table. The first row (`large_y_counts`) is the number of large Y vaues by group. The second (`small_y_counts`) is the number of small y values per group.

In [ ]:
large_y_counts = []
small_y_counts = []
Y_med = np.median(Y)
for g in ['A','B','C']:
    large_y_count = 0
    small_y_count = 0
    # Instead of looping through the values, we loop through the index.
    # That way we can also get the index of the `groups` variable
    for i in range(len(Y)):
        if group[i] == g:
            if Y[i] > Y_med:
                large_y_count += 1
            else:
                small_y_count += 1
    large_y_counts.append(large_y_count)
    small_y_counts.append(small_y_count)

In [197]:
# Like many things, this could be done more quickly with pandas
df['large_y'] = df.Y > df.Y.median()

large_y_counts = df.loc[df.large_y==True,:].groupby('group').Y.count()
small_y_counts = df.loc[df.large_y==False,:].groupby('group').Y.count()

In [198]:
# Now, we call the Chi-squared test
stats.chi2_contingency(np.array([large_y_counts, small_y_counts]))
# This returns the Chi-square value, a p-value, degrees of freedom, and the expected counts.

(12.32280701754386, 0.0021092907851615445, 2, array([[19., 15., 16.],
        [19., 15., 16.]]))

### ANOVA

This tests whether the means of multiple groups have the same population mean.

In [85]:
# these all should
stats.f_oneway(X1,X2,X3)

F_onewayResult(statistic=0.20072298916835568, pvalue=0.8182499263023821)

In [86]:
# but adding Y should change it

stats.f_oneway(X1,X2,X3,Y)

F_onewayResult(statistic=15.33062583228185, pvalue=1.8662451407414083e-09)

### Linear Regression

In [90]:
# Simple linear regression is possible with scipy stats
stats.linregress(X1, Y)

LinregressResult(slope=1.8108511287523748, intercept=1.552793497695232, rvalue=0.46809993053820487, pvalue=9.0628093323292e-07, stderr=0.34532177628953775)

### Multiple Linear Regression

In [100]:
# but for multiple regression we need to use something else. One option is sklearn,
# the machine learning package. Another, maybe simpler is statsmodels, which I show here:

import pandas as pd
import statsmodels.formula.api as sm

In [97]:
df = pd.DataFrame({'X1':X1,
                   'X2':X2,
                   'X3': X3,
                   'group':group,
                   'Y':Y})

In [102]:
result = sm.ols(formula="Y ~ X1 + X2 + X3 + group", data=df).fit()

In [105]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.920
Method:                 Least Squares   F-statistic:                     229.3
Date:                Mon, 13 Apr 2020   Prob (F-statistic):           4.89e-51
Time:                        17:04:58   Log-Likelihood:                -138.22
No. Observations:                 100   AIC:                             288.4
Df Residuals:                      94   BIC:                             304.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.1605      0.180     17.557      0.000       2.803       3.518
group[T.B]    -1.7978      0.238     -7.564      0.000      -2.270      -1.326
group[T.C]    -3.0180      0.264    -11.431      0.000      -3.542      -2.494
X1             1.5869      0.111     14.293      0.000       1.366       1.807
X2            -2.4333      0.092    -26.431      0.000      -2.616      -2.250
X3            -0.1049      0.088     -1.196      0.235      -0.279       0.069
==============================================================================
Omnibus:                        4.760   Durbin-Watson:                   2.380
Prob(Omnibus):                  0.093   Jarque-Bera (JB):                4.472
Skew:                           0.518   Prob(JB):                        0.107
Kurtosis:                       3.043   Cond. No.                         4.10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Note the benefit of regression - the coefficient for X1 is much closer to the true coefficient (1.5)